# Configuration

In [ ]:
drop_locality_data = False
run_knn = False
run_knn_hyperparameter_error_plot = False
save_figs = False

# Initialization

In [ ]:
import pytz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, classification_report, confusion_matrix
from datetime import datetime, timedelta
%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

plt.rcParams.update(
    {
        'font.size': 18,
        'savefig.dpi': 150,
        'xtick.labelsize': 14,
        'ytick.labelsize': 14
    }
)


households_df = pd.read_csv('./ITAASER2019HOUSEHOLD.csv', dtype={
    'AREA': 'object',
    'VCODE': 'object',
    'HHID': 'object',
    'H002': 'object',
})

households_df = households_df.astype({
    'H003': 'bool',
    'H004': 'bool',
    'H005': 'bool',
    'H006': 'bool',
    'H007': 'bool',
    'H008': 'bool',
    'H009': 'bool',
    'H010': 'bool',
    'H011': 'bool',
    'H012': 'bool',
    'HEHSAAS': 'bool',
    'HBISP': 'bool',
    'HPSPA': 'bool',
    'HAKHU': 'bool',
    'HSPO': 'bool',
    'HSPNON': 'bool',
    'H100': 'bool',
    'H101': 'bool'

})

households_df = households_df.rename(columns={
    'AREA': 'area',
    'H001M': 'num_males',
    'H001F': 'num_females',
    'H001': 'num_members',
    'VCODE': 'district_code',
    'H002': 'house_type',
    'H003': 'is_self_owned',
    'H004': 'has_electric_connection',
    'H005': 'is_television_available',
    'H006': 'is_mobile_phone_available',
    'H007': 'is_smart_phone_available',
    'H008': 'is_car_available',
    'H009': 'is_motorcycle_available',
    'H010': 'is_computer_available',
    'H011': 'is_solar_panel_available',
    'H012': 'is_internet_connection_available',
    'HEHSAAS': 'is_ehsaas_recepient',
    'HBISP': 'is_bisp_recepient',
    'HPSPA': 'is_pspa_recepient',
    'HAKHU': 'is_akhuwat_recepient',
    'HSPO': 'is_other_ssn_recepient',
    'HSPNON': 'is_ssn_recepient',
    'H100': 'uses_sms',
    'H101': 'uses_whatsapp'

})

households_df['house_type'] = households_df['house_type'].map({
    '1': 'kutcha',
    '2': 'semi-pucca',
    '3': 'pucca'
})


parents_df = pd.read_csv('./ITAASER2019PARENT.csv', dtype={
    'AREA': 'object',
    'VCODE': 'object',
    'PR005': 'object',
    'PR010': 'object',
    'PRID': 'object'
    # Create a column for years of education
})

parents_df = parents_df.astype({
    'PR004': 'bool',
    'PR006': 'bool',
    'PR009': 'bool',
    'PR011': 'bool',
})

parents_df = parents_df.rename(columns={
    'AREA': 'area',
    'VCODE': 'district_code',
    'PR001': 'mother_age',
    'PR002': 'num_surviving_children_younger_than_17',
    'PR003': 'num_surviving_children_elder_than_17',
    'PR004': 'has_mother_ever_attended_school',
    'PR005': 'mother_max_grade',
    'PR006': 'is_mother_education_more_than_5_grades',
    'PR009': 'has_father_ever_attended_school',
    'PR010': 'father_max_grade',
    'PR011': 'is_father_education_more_than_5_grades',
})




parents_df = parents_df.drop(columns=['num_surviving_children_younger_than_17', 'num_surviving_children_elder_than_17'])


children_df = pd.read_csv('./ITAASER2019CHILD.csv', dtype={
    'AREA': 'object',
    'VCODE': 'object',
    'PRID': 'object',
    'CID': 'object',
    'HHID': 'object',
    'C002': 'object',
    'C003': 'object',
    'C003a': 'object',
    'C003b': 'object',
    'C004': 'object',
    'C005': 'object',
    'C006': 'object',
    'C010': 'object',
    'C011': 'object',
    'C012': 'object',
    'C013': 'object',
})




children_df = children_df.astype({
    'C008a': 'bool',
    'C010a': 'bool',
    'C010b': 'bool',
    'C012a': 'bool',
    'C012b': 'bool',
    'C012c': 'bool',
    'C014': 'bool',
    'C015': 'bool',
    'C016': 'bool',
    'C017': 'bool',
    'C018': 'bool',
    'C019': 'bool',    
})

children_df = children_df.rename(columns={
    'AREA': 'area',
    'VCODE': 'district_code',
    'C001': 'age',
    'C002': 'gender',
    'C003': 'educational_status',
    'C003a': 'reason_for_dropout',
    'C003b': 'reason_for_non_enrollment',
    'C004': 'max_grade_if_dropout',
    'C005': 'current_grade',
    'C006': 'current_institution_type',
    'C007': 'is_enrolled_in_survey_school',
    'C008a': 'is_taking_tuition_currently',
    'C008b': 'current_tuition_fee',
    'C010': 'basic_learning_level',
    'C010a': 'basic_learning_level_q1_correct',
    'C010b': 'basic_learning_level_q1_correct',
    'C011': 'basic_learning_level_test_language',
    'C012': 'arithmetic_levels',
    'C012a': 'arithmetic_level_word_problem_1_correct',
    'C012b': 'arithmetic_level_word_problem_2_correct',
    'C012c': 'arithmetic_level_word_problem_3_correct',
    'C013': 'english_reading_level',
    'C013a': 'english_reading_level_bonus_question_1_correct',
    'C013b': 'english_reading_level_bonus_question_2_correct',
    'C014': 'english_knows_word_meanings',
    'C015': 'english_knows_sentence_meanings',
    'C016': 'general_knowledge_knows_english_poem',
    'C017': 'general_knowledge_english_q2_correct',
    'C018': 'general_knowledge_english_figures_recognition',
    'C019': 'is_child_present_in_household'
})

children_df['reason_for_dropout'] = children_df['reason_for_dropout'].map({
    '1': 'Law and Order',
    '2': 'Poverty',
    '3': 'Flood',
    '4': 'School building shifted by govt.',
    '5': 'Illness',
    '6': 'Migration',
    '7': 'Others'
})

children_df['reason_for_non_enrollment'] = children_df['reason_for_non_enrollment'].map({
    '1': 'Law and Order',
    '2': 'Poverty',
    '3': 'Flood',
    '4': 'School building shifted by govt.',
    '5': 'Illness',
    '6': 'Migration',
    '7': 'Others'
})

children_df['is_currently_enrolled'] = children_df['educational_status'] == '3'


enrolled_children_df = children_df[children_df['educational_status'] == '3']
dropout_children_df = children_df[children_df['educational_status'] == '2']
never_enrolled_children_df = children_df[children_df['educational_status'] == '1']

print(enrolled_children_df.shape)
print(dropout_children_df.shape)
print(never_enrolled_children_df.shape)


schools_df = pd.read_csv('./ITAASER2019SCHOOLS.csv')

# Helper functions

In [ ]:
def map_max_grade_to_years_of_education(df, col_name):
    return df[col_name].map({
     '1': '1',
     '2': '2',
     '3': '3',
     '4': '4',
     '5': '5',
     '6': '6',
     '7': '7',
     '8': '8',
     '9': '9',
     '10': '10',
     'Alim/Hafiz/Madrassah': '10',
     'B.Ed/M.Ed': '14',
     'BACHELORS': '14',
     'CT': '14',
     'GRADUATE': '14',
     'INTERMEDIATE': '12',
     'M PHIL': '18',
     'MASTERS': '16',
     'MPHIL': '18',
     'PHD': '20'
    }).astype('float').astype('Int64')

# Households

In [ ]:
print(households_df.shape)
households_df.head()

# Schools

In [ ]:
print(schools_df.shape)
schools_df.head()

# Children

In [ ]:
print(children_df.shape)
children_df.head()

# Parents

In [ ]:
print(parents_df.shape)
parents_df.head()

# Respondents' Opinion - Dropping Out

In [ ]:
dropout_reasons_grouped_df = children_df[children_df['RNAME'] != 'ISLAMABAD - ICT'].groupby(['RNAME', 'reason_for_dropout'], as_index=False).count()
rname_fill_counts = children_df.dropna(subset=['reason_for_dropout']).RNAME.value_counts()

percentage_dropout_reasons_grouped_df = dropout_reasons_grouped_df
percentage_dropout_reasons_grouped_df['total_fills'] = percentage_dropout_reasons_grouped_df['RNAME'].apply(lambda x: rname_fill_counts[x])
percentage_dropout_reasons_grouped_df['percentage_fills'] = (percentage_dropout_reasons_grouped_df['CID'] / percentage_dropout_reasons_grouped_df['total_fills'])*100



pivot_df = pd.pivot_table(percentage_dropout_reasons_grouped_df,'percentage_fills','RNAME','reason_for_dropout')

plt.subplots(figsize=(20, 12))
ax = sns.heatmap(pivot_df, annot=True, cmap="YlGnBu", fmt='.1f')
print(f'Total responses: {np.sum(rname_fill_counts)}')
plt.title('Respondents Opinion - Dropout Reasons', fontsize=20)
if save_figs:
    plt.savefig('./figures/respondents_opinion_dropout_reasons')

In [ ]:
dropout_reasons_grouped_df = children_df.groupby(['DNAME', 'reason_for_dropout'], as_index=False).count()
rname_fill_counts = children_df.dropna(subset=['reason_for_dropout']).DNAME.value_counts()

percentage_dropout_reasons_grouped_df = dropout_reasons_grouped_df
percentage_dropout_reasons_grouped_df['total_fills'] = percentage_dropout_reasons_grouped_df['DNAME'].apply(lambda x: rname_fill_counts[x])
percentage_dropout_reasons_grouped_df['percentage_fills'] = (percentage_dropout_reasons_grouped_df['CID'] / percentage_dropout_reasons_grouped_df['total_fills'])*100



pivot_df = pd.pivot_table(percentage_dropout_reasons_grouped_df,'percentage_fills','DNAME','reason_for_dropout')

plt.subplots(figsize=(10, 60))
ax = sns.heatmap(pivot_df, annot=True, cmap="YlGnBu", fmt='.1f')
print(f'Total responses: {np.sum(rname_fill_counts)}')
plt.title('Respondents Opinion - Dropout Reasons', fontsize=20)
plt.savefig('./figures/respondents_opinion_dropout_reasons')

# Respondents' Opinion - Non Enrollment

In [ ]:
non_enrollment_reasons_grouped_df = children_df.groupby(['RNAME', 'reason_for_non_enrollment'], as_index=False).count()
rname_fill_counts = children_df.dropna(subset=['reason_for_non_enrollment']).RNAME.value_counts()

percentage_dropout_reasons_grouped_df = non_enrollment_reasons_grouped_df
percentage_dropout_reasons_grouped_df['total_fills'] = percentage_dropout_reasons_grouped_df['RNAME'].apply(lambda x: rname_fill_counts[x])
percentage_dropout_reasons_grouped_df['percentage_fills'] = (percentage_dropout_reasons_grouped_df['CID'] / percentage_dropout_reasons_grouped_df['total_fills'])*100

pivot_df = pd.pivot_table(percentage_dropout_reasons_grouped_df,'percentage_fills','RNAME','reason_for_non_enrollment')

plt.subplots(figsize=(20, 12))
ax = sns.heatmap(pivot_df, annot=True, cmap="YlGnBu", fmt='.2f')

print(f'Total Responses: {np.sum(rname_fill_counts)}')

plt.title('Respondents Opinion - Non Enrollment Reasons', fontsize=20)
if save_figs:
    plt.savefig('./figures/respondents_opinion_non_enrollment_reasons')

# Finding out the main factors for enrollment

In [ ]:
children_households_df = pd.merge(households_df,children_df, on='HHID', suffixes=('_hh','_ch'))
children_parents_df = pd.merge(children_df, parents_df, on='PRID')
children_households_parents_df = pd.merge(children_households_df, parents_df, on='PRID', suffixes=('_chh', '_pr'))

# Remove columns that give us context on a child's education. 
# Wouldn't be a useful predictive model if it needs information about a child's
# performance in language and arithmetic questions
trimmed_children_households_parents_df = children_households_parents_df.drop(columns=['SYEAR_ch','area_ch','RNAME_ch','DNAME_ch',
                                                                     'district_code_ch', 'HCOUNTER_ch','reason_for_dropout',
                                                                     'reason_for_non_enrollment','max_grade_if_dropout',
                                                                     'current_grade', 'current_institution_type',
                                                                     "C020", "is_taking_tuition_currently", 
                                                                      "current_tuition_fee", "basic_learning_level", 
                                                                      "basic_learning_level_q1_correct", 
                                                                      "basic_learning_level_q1_correct", 
                                                                      "basic_learning_level_test_language", 
                                                                      "arithmetic_levels", "arithmetic_level_word_problem_1_correct",
                                                                      "arithmetic_level_word_problem_2_correct", 
                                                                      "arithmetic_level_word_problem_3_correct", "english_reading_level",
                                                                      "english_knows_word_meanings", "english_knows_sentence_meanings",
                                                                      "general_knowledge_knows_english_poem", "general_knowledge_english_q2_correct",
                                                                      "general_knowledge_english_figures_recognition", 
                                                                      "is_child_present_in_household", "HHID_pr", "HCOUNTER",
                                                                      "PR008", "district_code_hh", "HCOUNTER_hh", "SYEAR_hh",
                                                                      "HHID_chh", "PRID", "CID", "area_hh", "district_code",
                                                                      "RNAME_hh", "DNAME_hh"
                                                                                     ])


trimmed_children_households_parents_df = trimmed_children_households_parents_df \
                                                        .dropna(subset=['gender', 'house_type', 'area'], how='any')

print(trimmed_children_households_parents_df.shape)
print(trimmed_children_households_parents_df.is_currently_enrolled.value_counts())
print(trimmed_children_households_parents_df.is_currently_enrolled.value_counts())

children_households_parents_df[children_households_parents_df['is_currently_enrolled']==False].isna().mean().sort_values(ascending=False)






# Imputing Values

In [ ]:
trimmed_children_households_parents_df.isna().mean().sort_values(ascending=False)

In [ ]:
fill_mean = lambda col: col.fillna(col.mean())

cols_to_impute = ['num_members', 'num_females', 'num_males', 'mother_age']

trimmed_children_households_parents_df[cols_to_impute] = trimmed_children_households_parents_df[cols_to_impute].apply(fill_mean, axis=0)

print(trimmed_children_households_parents_df.isna().mean().sort_values())
trimmed_children_households_parents_df.head()

# How parents influence the enrollment of children in school?

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
corr_columns = ['has_electric_connection', 'is_television_available']
sns.heatmap(
    trimmed_children_households_parents_df.corr(), 
    ax=ax, 
    annot=True, 
    annot_kws={"size": 8}, 
    fmt='.2f', 
    cmap="coolwarm",
    vmin=-1,
    vmax=1)

trimmed_children_households_parents_df['has_mother_ever_attended_school'].value_counts()

if save_figs:
    plt.savefig('./figures/data_correlation', dpi=300, bbox_inches='tight')

In [ ]:
def encode_cat_col(df, col_name, dummy_na=True):
    encoded_df = pd.get_dummies(df[col_name], prefix=col_name, dummy_na=dummy_na)
    return pd.concat([df.drop(col_name, axis=1), encoded_df], axis=1)

if drop_locality_data:
    encoded_df = trimmed_children_households_parents_df.drop(columns=['RNAME','DNAME','area'])
else:
    encoded_df = encode_cat_col(trimmed_children_households_parents_df, 'RNAME')
    encoded_df = encode_cat_col(encoded_df, 'DNAME')


encoded_df = encode_cat_col(encoded_df, 'house_type')
encoded_df = encode_cat_col(encoded_df, 'mother_max_grade')
encoded_df = encode_cat_col(encoded_df, 'father_max_grade')
encoded_df = encode_cat_col(encoded_df, 'gender')
encoded_df = encode_cat_col(encoded_df, 'area')

print(encoded_df.shape)
encoded_df.head()
encoded_df.isna().mean()

In [ ]:
endoded_df_sample = encoded_df #.sample(n=50000, random_state=1)

X = endoded_df_sample.drop(columns=[
    'educational_status',
    'is_currently_enrolled'
])
y = endoded_df_sample['educational_status']



if run_knn:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=30)

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors=16, n_jobs=-1)
    classifier.fit(X_train, y_train)


    y_pred = classifier.predict(X_test)

    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    from sklearn import metrics
    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    
    cf_matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(cf_matrix, annot=True)



In [ ]:
error = []

# Calculating error for K values between 1 and 40

if run_knn_hyperparameter_error_plot:
    for i in range(1, 40, 1):
        print(f'Start knn with k={i}')
        knn = KNeighborsClassifier(n_neighbors=i, n_jobs=-1)
        knn.fit(X_train, y_train)
        pred_i = knn.predict(X_test)
        error.append(np.mean(pred_i != y_test))

        from sklearn.metrics import classification_report, confusion_matrix
        print(confusion_matrix(y_test, pred_i))
        print(classification_report(y_test, pred_i))

        print(f'error:{np.mean(pred_i != y_test)}')




    plt.figure(figsize=(12, 6))
    plt.plot(range(1,40,1), error, color='red', linestyle='dashed', marker='o',
             markerfacecolor='blue', markersize=10)
    plt.title('Error Rate K Value')
    plt.xlabel('K Value')
    plt.ylabel('Mean Error')

How Location/Districts contribute to children being in school


In [ ]:
from sklearn.linear_model import LogisticRegression

print(encoded_df.shape)
X = encoded_df.drop(columns=[
    'educational_status',
    'is_currently_enrolled'
])
y = encoded_df['is_currently_enrolled']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=30)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

lr = LogisticRegression(random_state=10, n_jobs=-1)
lr.fit(X_train, y_train)


y_pred = lr.predict(X_test)

from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True, fmt='.2f')


# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

scaler.transform(X_train)

In [ ]:



def coef_weights(coefficients, X_train):
    '''
    INPUT:
    coefficients - the coefficients of the linear model 
    X_train - the training data, so the column names can be used
    OUTPUT:
    coefs_df - a dataframe holding the coefficient, estimate, and abs(estimate)
    
    Provides a dataframe that can be used to understand the most influential coefficients
    in a linear model by providing the coefficient estimates along with the name of the 
    variable attached to the coefficient.
    '''
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X.columns
    coefs_df['coefs'] = coefficients
    coefs_df['abs_coefs'] = np.abs(coefficients)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    return coefs_df



coef_df = coef_weights(lr.coef_[0], X_train)

positive_factors_df = coef_df[coef_df['coefs'] > 0]
negative_factors_df = coef_df[coef_df['coefs'] < 0]

coef_df.head()


# What keeps children in school

In [ ]:
ax = positive_factors_df.iloc[0:20].plot(
    kind='bar', 
    x='est_int', 
    y='abs_coefs', 
    figsize=(15,8), 
    title='Top 20 positive factors')
ax.set_xlabel('Factors', fontsize=16)
if save_figs:
    plt.savefig('./figures/positive_factors_w_location.png', bbox_inches='tight')

# What makes children withdraw from school

In [ ]:
ax = negative_factors_df.iloc[0:20].plot(
        kind='bar', 
        x='est_int', 
        y='coefs', 
        figsize=(15,8), 
        color='maroon', 
        title='Top 20 negative factors')
ax.set_xlabel('Factors', fontsize=16)

if save_figs:
    plt.savefig('./figures/negative_factors_with_location.png', bbox_inches='tight')

# How Aid affects enrollment

In [ ]:
ssn_list = ['is_pspa_recepient', 'is_ssn_recepient', 'is_akhuwat_recepient', 'is_ehsaas_recepient', 'is_other_ssn_recepient', 'is_bisp_recepient']
coef_df[coef_df['est_int'].isin(ssn_list)].plot(kind='bar', x='est_int', y='coefs', figsize=(5,5))


# Age (In School vs Out of School)

In [ ]:
in_school_df = encoded_df[encoded_df['is_currently_enrolled']]
out_of_school_df = encoded_df[encoded_df['is_currently_enrolled'] != True]

In [ ]:
plt.figure(figsize=(20,8))
titles = ['In School', 'Out of School']
colors = [None, 'maroon']
for idx, df in enumerate([in_school_df, out_of_school_df]):
    plt.subplot(1, 2, idx+1)
    plt.title(titles[idx])
    plt.ylabel('No. of children')
    plt.hist(df.age, histtype= 'bar', color=colors[idx])

In [ ]:
sns.set(style="whitegrid")

in_and_out_of_school = pd.concat(
    [in_school_df['age'].value_counts().rename('in_school'),
     encoded_df['age'].value_counts().rename('total')], axis=1)

f,ax = plt.subplots(figsize=(20, 10))

sns.set_color_codes("pastel")
sns.barplot(x="index", y="total", data=in_and_out_of_school.reset_index(),
            label="Total Children", color="b")

sns.set_color_codes("muted")
sns.barplot(x="index", y="in_school", data=in_and_out_of_school.reset_index(),
            label="In School", color="b")

ax.tick_params(labelsize=15)
ax.legend(ncol=2, loc="upper right", frameon=True)
ax.set(xlim=(-1, 14), ylabel="",
       xlabel="Enrollments based on Age")
sns.despine(left=True, bottom=True)

if save_figs:
    plt.savefig('./figures/barplot.png')

In [ ]:
coef_df

# How parents' education affects children's education

In [ ]:
mother_father_attendance = encoded_df[['has_mother_ever_attended_school', 'has_father_ever_attended_school', 'is_currently_enrolled']].groupby(['has_mother_ever_attended_school', 'has_father_ever_attended_school']).count()

pivot_df = pd.pivot_table(mother_father_attendance,'is_currently_enrolled','has_mother_ever_attended_school','has_father_ever_attended_school')

plt.subplots(figsize=(5, 5))
plt.title('How parents\' enrollment affects child\'s chances')
ax = sns.heatmap(pivot_df, annot=True, cmap="YlGnBu", fmt='.2f')


In [ ]:
children_households_parents_df['father_years_of_education'] = map_max_grade_to_years_of_education(children_households_parents_df, 'father_max_grade'
children_households_parents_df['mother_years_of_education'] = map_max_grade_to_years_of_education(children_households_parents_df, 'mother_max_grade')

parents_max_grade_df = children_households_parents_df.groupby(['father_years_of_education', 'mother_years_of_education'])['is_currently_enrolled'].value_counts(normalize=True) * 100

parents_max_grade_df = pd.DataFrame(parents_max_grade_df)
parents_max_grade_df = parents_max_grade_df.unstack(2)

parents_max_grade_df['enrolled_percentage'] = parents_max_grade_df[[('is_currently_enrolled', True)]]

pivot_df = pd.pivot_table(parents_max_grade_df, 'enrolled_percentage','father_years_of_education', 'mother_years_of_education')
pivot_df = pivot_df.sort_index(ascending=False)

plt.subplots(figsize=(20, 15))
plt.title('How parents\' years of education affects child\'s enrollment percentages', fontsize=18, pad=30)

ax = sns.heatmap(pivot_df, annot=True, cmap="YlGnBu", fmt='.0f')
ax.tick_params(labelsize=18)
ax.set(xlabel='mother_years_of_education')
plt.yticks(rotation=0)  
plt.xticks(rotation=270)  
if save_figs:
    plt.savefig('./figures/parents_education_vs_child_education.png')
# pivot_df
